# Variability in performance on the *same* puzzle

The [performance](Performance.ipynb) of the constriant propogation algorithm shows a tail of test cases with very long solution times. The majority of the test puzzles are solved in < ~ 0.2 seconds, however there are some that take longer, up to a maxiumum of 12 seconds.

While looking into *why* this happened I found that sometimes I'd get different performance times *for the same puzzle*. Wildly different performance times. Things weren't as *deterministic* as I'd assumed they'd be. So I looked into it and eventually found the answer below.

**TL;DR: Python sets do not have a guaranteed order, even for sets of small integers**

## Modules required

We're using the [sudoku](../puzzle/sudoku.py) and [tester](../puzzle/tester.py) modules used elsewhere, as well as a small number of standard libraries. We have to make a slight adjustment to the notebook's environment in order to find these modules, since this notebook is in a sub-directory.

In [1]:
import sys
sys.path.insert(-1, '..')

import puzzle.tester as tester
import puzzle.sudoku as su
from puzzle.jupyter_helpers import *
display(HTML(SUDOKU_CSS))

# Puzzle 46

I'd been looking at the [performance](Performance.ipynb) of the different algorithms and why "slow" algorithms like backtracking were sometimes faster than better performers like constraint propogation. I was using different puzzles in [top95.txt](../data/sudoku_9x9/top95.txt), for example this one called `puzzle46` (it's line 46 in the file). From the previous test run, I expected this to take about 2.5 seconds.


In [5]:
puzzle46 = '.834.........7..5...........4.1.8..........27...3.....2.6.5....5.....8........1..'
puzzle = su.SudokuPuzzle(starting_grid=su.from_string(puzzle46))
method = 'constraintpropogation'
solver = su.SudokuSolver(method=method)
%time solver.solve(puzzle)

CPU times: user 2.55 s, sys: 9.57 ms, total: 2.56 s
Wall time: 2.6 s


True

So that *normally* takes 2.5 seconds on my 2017 i7 Mac. But sometimes it finished in ~0.3 seconds. It took a while to figure out how to reproduce the behaviour -- in the end, it's repetition that's required.

Let's run it again...and again. And this time we'll output some of the stats collected by the solver itself:

* Max Depth: The deepest stack-depth the recursion reached
* Backtracks: The number of "mistakes" made by the algorithm that resulted in it backtracking to a prior state


In [6]:
puzzle = su.SudokuPuzzle()
solver = su.SudokuSolver(method=method)
for i in range(5):
    print(f"RUN {i}")
    puzzle.init_puzzle(starting_grid=su.from_string(puzzle46))
    %time solver.solve(puzzle)
    print(f"Max Depth={solver.solver.max_depth}; Backtracks={solver.solver.backtrack_count}\n")

RUN 0
CPU times: user 2.56 s, sys: 12.7 ms, total: 2.58 s
Wall time: 2.63 s
Max Depth=63; Backtracks=105917

RUN 1
CPU times: user 2.52 s, sys: 7.24 ms, total: 2.52 s
Wall time: 2.55 s
Max Depth=63; Backtracks=105917

RUN 2
CPU times: user 220 ms, sys: 559 µs, total: 220 ms
Wall time: 220 ms
Max Depth=63; Backtracks=8766

RUN 3
CPU times: user 246 ms, sys: 2.95 ms, total: 249 ms
Wall time: 256 ms
Max Depth=63; Backtracks=8766

RUN 4
CPU times: user 221 ms, sys: 654 µs, total: 222 ms
Wall time: 222 ms
Max Depth=63; Backtracks=8766



In testing, the first two usually take ~2.5 seconds, and the last 3 take ~220 ms. Even though it's exactly the same puzzle being solved by exactly the same algorithm.

# Finding out why

The first clue as to why the performance time changes is in the backtracks count. The longer time has the higher backtrack count (~106K vs ~9K), exactly the same proportional difference that is reflected in the times.

So *sometimes*, the backtracking algorithm took a longer search path. But if it's the same puzzle and the same algorithm, why does it take a different path?

It turns out the difference is caused by the return order of the set of allowed values. When choosing which value to write into a cell, I call a method `get_allowed_values` on that cell. It returns the values as a Python `set`. In Python, the [order of the elements in a set is not guaranteed to be consistent](https://stackoverflow.com/questions/3812429/is-pythons-set-stable).  In this specific case, one of the cells has allowed values (1, 9). If "1" is chosen then the solution takes 2.5 seconds to find. However if the "9" is chosen, then the search takes ~220 ms. 

If I modify the algorithm to always sort the allowed values then the performance time is always consistent. Otherwise there's a certain amount of variability when solving some of the puzzles.

There are still some things I don't understand (well, there are a *lot* of things I don't understand, but I mean in this context...)

* If I recreate a new `puzzle` instance each time the performance remains consistent, because the set order remains consistent. But why?
* I can't get the set order to change in smaller examples.
* For "small integers" the hash used in the set is basically the value of the integer -- so why does the order change? I understand I'm not *supposed* to rely on it, but it's still surprising that it changes.

[Peter Norvig encountered a similar question](https://norvig.com/sudoku.html) although he uses strings and not sets/lists. In his code the order of "allowed values" does not change unexpectedly, but he still observes that sometimes performance degrades considerably because the algorithm chooses a wrong value early and it takes a long time to backtrack and try a new path. From his article:

> My guess is that somewhere early in the search there is a sequence of squares (probably two) such that if we choose the exact wrong combination of values to fill the squares, it takes about 190 seconds to discover that there is a contradiction. But if we make any other choice, we very quickly either find a solution or find a contradiction and move on to another choice. So the speed of the algorithm is determined by whether it can avoid the deadly combination of value choices. 
>
> *[Solving every Sudoku Puzzle](https://norvig.com/sudoku.html), Peter Norvig*


# What can we do about it?

I'm not even sure if I *should* do anything about it. This was just some "random" behaviour that was puzzling, but it's not "wrong". Here are the options:

1. Do nothing.
2. Fix the ordering of allowed values deterministically (e.g. sort by value).
3. Set the [PYTHONHASHSEED](https://docs.python.org/3.3/using/cmdline.html#envvar-PYTHONHASHSEED) environment, which [should trigger deterministic hashing behavior](https://stackoverflow.com/a/32529871/2997).
3. *Deliberately* randomize the value ordering, since it seems these "deadly combinations" are relatively rare, so if we run repeated samples then most runs should be OK.
4. Choose the value to guess more deliberately -- are there "smarter" choices which have the effect of reducing the number of unsuccessful search paths?
5. Detect if the algorithm is taking an "unusually long time" to solve a puzzle, and abort. Then retry from scratch, with randomized selection of values for trying in a cell. When the backtracking solution works it can work pretty quickly, so "giving up" and trying again may actually be faster than repeatedly trying variations of a bad search path.

The last two suggestions come from Norvig (Ibid.) -- he tried the randomization path and found some success in reducing the incidence of these "really slow" performance times. He suggests the fourth approach, such as choosing the "[least-constraining value](https://cs.stackexchange.com/questions/47870/what-is-least-constraining-value)," but didn't evaluate it in that article.
